In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

# TOP-500 фильмов с КиноПоиска
Списки на Кинопоиске оформлены абсолютно одинаково, поэтому просто применим код, который мы использовали для ТОПа-100.

In [7]:
# загрузим файлы

soups = []
for i in range(10):
    with open(str(i) + '.txt', 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
        soups.append(soup)

In [8]:
# создадим списки с нужными блоками кода.

infos = []
ratings_raw = []

for soup in soups:
    infos += soup.findAll('div', {'class': 'styles_main__Y8zDm'})
    ratings_raw += soup.findAll('div', {'class': 'styles_kinopoisk__JZttS'})

In [11]:
# соберем все данные в списки

ru_names = []
en_names = []
years = []
lengths = []
countries = []
genres = []
directors = []
casts = []
ratings = []
ratings_numbers = []

for info, raw_rating in zip(infos, ratings_raw):
    ru_name = info.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text
    is_en_name = info.find('span', {'class': 'desktop-list-main-info_secondaryTitle__ighTt'})
    if is_en_name:
        en_name = is_en_name.text
    else:
        en_name = ''
    
    length_n_year = info.find('span', {'class': 'desktop-list-main-info_secondaryText__M_aus'}).text
    year = int(re.findall(r'\d{4}', length_n_year)[0])
    length = int(''.join(re.findall(r'\d', re.findall(r'\d{2,3}\xa0мин.', length_n_year)[0])))

    other1 = info.findAll('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'})[0]
    country = re.findall('.+ •', other1.text)[0][:-2]
    genre = re.findall('• .+\xa0\xa0', other1.text)[0][2:-2]
    director = re.findall('Режиссёр: .+', other1.text)[0][10:]
    is_two = len(info.findAll('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}))
    if is_two == 2:
        other2 = info.findAll('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'})[1]
        cast = re.findall('В ролях: .+', other2.text)[0][9:].split(', ')
    else:
        cast = []
        
    rating = float(re.findall(r'\d{1}.\d{1}', raw_rating.find('span', {'class': 'styles_srRoot__WgbFG'}).text)[0])
    ratings_number = int(''.join(re.findall(r'\d', raw_rating.findAll('span', {'class': 'styles_srRoot__WgbFG'})[1].text)))
    
    ru_names.append(ru_name)
    en_names.append(en_name)
    years.append(year)
    lengths.append(length)
    countries.append(country)
    genres.append(genre)
    directors.append(director)
    casts.append(cast)
    ratings.append(rating)
    ratings_numbers.append(ratings_number)

In [12]:
# зададим функцию, которая будет собирать списки в датафрейм

def df_templater():
    return {'ru_name': ru_names,
           'en_name': en_names,
           'year': years,
           'length': lengths,
           'country': countries,
           'genre': genres,
           'director': directors,
           'cast': casts,
           'rating': ratings,
           'ratings_number': ratings_numbers}

In [14]:
# соберем датафрейм

top500_kp = pd.DataFrame(df_templater())
top500_kp.head()

,ru_name,en_name,year,length,country,genre,director,cast,rating,ratings_number
0,Зеленая миля,The Green Mile,1999,189,США,драма,Фрэнк Дарабонт,"[Том Хэнкс, Дэвид Морс]",9.1,114573
1,1+1,Intouchables,2011,112,Франция,драма,Оливье Накаш,"[Франсуа Клюзе, Омар Си]",9.0,442253
2,Побег из Шоушенка,The Shawshank Redemption,1994,142,США,драма,Фрэнк Дарабонт,"[Тим Роббинс, Морган Фриман]",9.0,119564
3,Форрест Гамп,Forrest Gump,1994,142,США,драма,Роберт Земекис,"[Том Хэнкс, Робин Райт]",9.0,109984
4,Интерстеллар,Interstellar,2014,169,США,фантастика,Кристофер Нолан,"[Мэттью Макконахи, Энн Хэтэуэй]",8.9,109489


In [16]:
# сохраним файл

with open('top500_kp.csv', 'w', encoding='utf-8', newline='') as f:
    f.write(top500_kp.to_csv())